In [2]:
import pandas as pd
import numpy as np
import os
from env import host, user, password

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression, RFE

pd.options.display.float_format = '{:20,.2f}'.format

In [3]:
import wrangle_zillow as wr

In [4]:
df = wr.get_zillow_data()
df.head()

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate
0,14297519,1727539,NaN,NaN,NaN,3.50,4.00,None,NaN,3.50,...,"1,023,282.00","2,016.00","537,569.00","11,013.72",None,NaN,"60,590,630,072,012.00",0,0.03,2017-01-01
1,17052889,1387261,NaN,NaN,NaN,1.00,2.00,None,NaN,1.00,...,"464,000.00","2,016.00","376,000.00","5,672.48",None,NaN,"61,110,010,023,006.00",1,0.06,2017-01-01
2,14186244,11677,NaN,NaN,NaN,2.00,3.00,None,NaN,2.00,...,"564,778.00","2,016.00","479,489.00","6,488.30",None,NaN,"60,590,218,022,012.00",2,0.01,2017-01-01
3,12177905,2288172,NaN,NaN,NaN,3.00,4.00,None,8.00,3.00,...,"145,143.00","2,016.00","36,225.00","1,777.51",None,NaN,"60,373,001,001,006.00",3,-0.10,2017-01-01
4,10887214,1970746,1.00,NaN,NaN,3.00,3.00,None,8.00,3.00,...,"119,407.00","2,016.00","45,726.00","1,533.89",None,NaN,"60,371,236,012,000.00",4,0.01,2017-01-01


In [7]:
pd.set_option("display.max_columns", None)

In [8]:
df.head()

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate
0,14297519,1727539,NaN,NaN,NaN,3.50,4.00,None,NaN,3.50,NaN,NaN,"3,100.00","3,100.00",NaN,NaN,NaN,NaN,"6,059.00",NaN,3.00,2.00,633.00,NaN,NaN,"33,634,931.00","-117,869,207.00","4,506.00",NaN,NaN,NaN,NaN,NaN,122,261.00,None,"60,590,630.07","53,571.00","1,286.00",NaN,"96,978.00",0.00,NaN,1.00,NaN,NaN,NaN,NaN,"1,998.00",NaN,NaN,"485,713.00","1,023,282.00","2,016.00","537,569.00","11,013.72",None,NaN,"60,590,630,072,012.00",0,0.03,2017-01-01
1,17052889,1387261,NaN,NaN,NaN,1.00,2.00,None,NaN,1.00,NaN,"1,465.00","1,465.00","1,465.00",NaN,NaN,"1,465.00",NaN,"6,111.00",1.00,1.00,1.00,0.00,NaN,NaN,"34,449,266.00","-119,281,531.00","12,647.00",NaN,NaN,NaN,NaN,NaN,1110,261.00,None,"61,110,010.02","13,091.00","2,061.00",NaN,"97,099.00",5.00,NaN,NaN,NaN,NaN,NaN,NaN,"1,967.00",1.00,NaN,"88,000.00","464,000.00","2,016.00","376,000.00","5,672.48",None,NaN,"61,110,010,023,006.00",1,0.06,2017-01-01
2,14186244,11677,NaN,NaN,NaN,2.00,3.00,None,NaN,2.00,NaN,NaN,"1,243.00","1,243.00",NaN,NaN,NaN,NaN,"6,059.00",NaN,2.00,2.00,440.00,NaN,NaN,"33,886,168.00","-117,823,170.00","8,432.00",1.00,NaN,NaN,NaN,1.00,122,261.00,None,"60,590,218.02","21,412.00","1,286.00",NaN,"97,078.00",6.00,NaN,NaN,NaN,NaN,NaN,NaN,"1,962.00",1.00,NaN,"85,289.00","564,778.00","2,016.00","479,489.00","6,488.30",None,NaN,"60,590,218,022,012.00",2,0.01,2017-01-01
3,12177905,2288172,NaN,NaN,NaN,3.00,4.00,None,8.00,3.00,NaN,NaN,"2,376.00","2,376.00",NaN,NaN,NaN,NaN,"6,037.00",NaN,3.00,NaN,NaN,NaN,2.00,"34,245,180.00","-118,240,722.00","13,038.00",1.00,NaN,NaN,NaN,1.00,0101,261.00,LCR110000*,"60,373,001.00","396,551.00","3,101.00",NaN,"96,330.00",0.00,NaN,NaN,NaN,1.00,NaN,NaN,"1,970.00",NaN,NaN,"108,918.00","145,143.00","2,016.00","36,225.00","1,777.51",None,NaN,"60,373,001,001,006.00",3,-0.10,2017-01-01
4,10887214,1970746,1.00,NaN,NaN,3.00,3.00,None,8.00,3.00,NaN,NaN,"1,312.00","1,312.00",NaN,NaN,NaN,NaN,"6,037.00",NaN,3.00,NaN,NaN,NaN,2.00,"34,185,120.00","-118,414,640.00","278,581.00",1.00,NaN,NaN,NaN,1.00,010C,266.00,LAR3,"60,371,236.01","12,447.00","3,101.00","268,548.00","96,451.00",0.00,NaN,NaN,NaN,1.00,NaN,NaN,"1,964.00",NaN,NaN,"73,681.00","119,407.00","2,016.00","45,726.00","1,533.89",None,NaN,"60,371,236,012,000.00",4,0.01,2017-01-01


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72241 entries, 0 to 72240
Data columns (total 62 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      72241 non-null  int64  
 1   id                            72241 non-null  int64  
 2   airconditioningtypeid         23079 non-null  float64
 3   architecturalstyletypeid      207 non-null    float64
 4   basementsqft                  50 non-null     float64
 5   bathroomcnt                   72241 non-null  float64
 6   bedroomcnt                    72241 non-null  float64
 7   buildingclasstypeid           0 non-null      object 
 8   buildingqualitytypeid         45080 non-null  float64
 9   calculatedbathnbr             71987 non-null  float64
 10  decktypeid                    612 non-null    float64
 11  finishedfloor1squarefeet      6032 non-null   float64
 12  calculatedfinishedsquarefeet  72055 non-null  float64
 13  f

In [10]:
def remove_columns(df, cols_to_remove):  
    df = df.drop(columns=cols_to_remove)
    return df

In [11]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .75):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [12]:
def data_prep(df, cols_to_remove=[], prop_required_column=.5, prop_required_row=.75):
    df = remove_columns(df, cols_to_remove)
    df = handle_missing_values(df, prop_required_column, prop_required_row)
    return df